In [1]:
import sys
sys.path.append('..')

import os
import os.path as osp
import time
import pickle
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TerminateOnNaN, TensorBoard
from sklearn.model_selection import train_test_split

# custom modules
from libs.utilities import load_dataset, create_autoencoder_model, load_autoencoder_model, load_autoencoder_lossfunc
from libs.model_utils import LossLayer
from libs.data_generator import DataGenerator
from libs.processing import pink_noise, s_to_power

# os.getcwd()

Using TensorFlow backend.
/home/christie/miniconda3/envs/denoising/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#### PARAMETERS ####

model_source = '../models/tcn.jsont'


In [3]:
#### OTHER VARS ####
# loss function: data slice under consideration
time_slice = slice(None)
initial_epoch = 0
input_shape = (256, 16, 1)
use_skip_connections = False
template_args = {"dropout_rate": 0.2, "channels": input_shape[2], "activ_func": "relu",
                 "timesteps": input_shape[1], "use_skip_connections": str(use_skip_connections).lower(),
                "ker_size":2, "dilatations": [1], "n_stacks": 1}


In [4]:
### SHOW MODEL FILE ###
import json
import pprint 

# pprint.pprint(json.load(open(model_source)))

In [5]:
### TRY AND GENERATE MODEL ###
# NOTE: restart the kernel and rerun everything after editing an imported script or lib
model, lossfunc = create_autoencoder_model(
    model_source, input_shape, template_args, time_slice=time_slice)

[u] Creating autoencoder model from ../models/tcn.jsont
[u] Model factory parameters: {'input_shape': (256, 16, 1), 'template_args': {'dropout_rate': 0.2, 'channels': 1, 'activ_func': 'relu', 'timesteps': 16, 'use_skip_connections': 'false', 'ker_size': 2, 'dilatations': [1], 'n_stacks': 1}, 'time_slice': slice(None, None, None)}
[m] Processing template...
[m] Adding layer PERMUTE  -  {'dims': [2, 1, 3]}
[m] Adding layer RESHAPE  -  {'target_shape': [16, -1]}
[m] Adding layer TCN  -  {'nb_filters': 256, 'dropout_rate': 0.2, 'kernel_size': 2, 'dilations': [1], 'nb_stacks': 1, 'use_skip_connections': False, 'return_sequences': True}
[m] Adding layer TCN  -  {'nb_filters': 256, 'dropout_rate': 0.2, 'kernel_size': 2, 'dilations': [1], 'nb_stacks': 1, 'use_skip_connections': False, 'return_sequences': True}
[m] Adding layer RESHAPE  -  {'target_shape': [16, -1, 1]}
[m] Adding layer PERMUTE  -  {'dims': [2, 1, 3]}


In [6]:
### VERIFY GENERATED MODEL ###
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 16, 1)    0                                            
____________________________________________________________________________________________________
permute_1 (Permute)              (None, 16, 256, 1)    0           input_1[0][0]                    
____________________________________________________________________________________________________
reshape_1 (Reshape)              (None, 16, 256)       0           permute_1[0][0]                  
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 16, 256)       65792       reshape_1[0][0]                  
___________________________________________________________________________________________

_____
# Can we use the built-in Keras Model exporters (dict, yaml, json)?

In [ ]:
### TEST KERAS BUILTIN JSON IMPORT/EXPORT
from models.model_example import AEModelFactory
obj = AEModelFactory(input_shape, 3, 16)
model2 = obj.get_model()

In [ ]:
model2.get_layer('encoder').summary()
model2.get_layer('decoder').summary()
model2.summary()


In [ ]:
from pprint import pprint
model_config = model2.get_config()
print(model_config)
#pprint(model_config)  ### looks prettier but takes all the space

In [ ]:
# well i guess that's a no

'false'